# Introducción a pyspark

## Instalación

Instalamos pyspark en nuestro entorno para ello utilizamos el gestor de paquetes **pip**.

para ello ejecutamos:

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


> El signo de admiración indica que la instrucción será ejecutada en consola.

## Creación de una sesión de Spark

Para empezar a trabajar con pyspark primero necesitamos crear una sesión de spark.

1. importamos el módulo **SparkSession**
2. Creamos la sesión **pysaprk_course**

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('pyspark_course').getOrCreate()
spark

In [ ]:
# !pip install pyspark
# import pyspark

# from pyspark.sql import SparkSession
# import pyspark.sql.functions as F
# spark = SparkSession.builder.appName('pysaprk_course').getOrCreate()
# spark

>**En el entorno DATIO de BBVA todas las acciones realizadas hasta ahora se realizan de manera automática, así que en DATIO no debemos preocuparnos ni de instalar la librería no crear la sesión de Spark.**

## Métodos básicos de pyspark


### Cargar datos en un Dataframe

#### `spark.read.csv`




Cuando leemos los datos de un csv hay que tener en cuenta dos cosas:   
    - cargar las cabeceras `header=True`  
    - inferir el tipo de dato de cada campo `inferSchema=True`

   Esto es así porque los csv son archivos de texto plano pero cuando se trata de archivos **parquet** (archivos binarios, que contienen metadatos) no es necesario especificar estas opciones `spark.read.parquet(path_to_file)`.

In [ ]:
df_tarjeta_credito_estudios = spark.read.csv('credit_card_education_level_v2.csv', header=True, inferSchema=True)
df_tarjeta_credito_info = spark.read.csv('credit_card_info_v3.csv', header=True,  inferSchema=True)


### Explorando el Dataframe

#### `printSchema()`

Nos muestra:

  - campos
  - data types

In [ ]:
df_tarjeta_credito_info.printSchema()

root
 |-- cliente: integer (nullable = true)
 |-- edad: integer (nullable = true)
 |-- genero: string (nullable = true)
 |-- sueldo: string (nullable = true)
 |-- limite de credito: double (nullable = true)



In [ ]:
df_tarjeta_credito_estudios.printSchema()

root
 |-- CLIENTNUM: integer (nullable = true)
 |-- Education_Level: string (nullable = true)



#### count()

Devuelve el número de registros del DF

In [ ]:
df_tarjeta_credito_info.count()

10125

In [ ]:
df_tarjeta_credito_estudios.count()

10125

#### `show()`


Por defecto imprime los primeros 20 registros del DF. 

Acepta 3 argumentos:

- n (int)         => número de registros a mostrar
- truncate (bool) => truncará el valor del campo en el caso que sea muy extenso
- vertical (bool) => mostrará los campos del DF en vertical

In [ ]:
df_tarjeta_credito_info.show(n=20,truncate=True,vertical=False)

+---------+--------------------+------------+------+---------------+--------------+---------------+-------------+--------------+---------------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|      Attrition_Flag|Customer_Age|Gender|Dependent_count|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_CoCLIENTNUMunt|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|Naive_Bayes_Classifier_Attrition_Flag

In [ ]:
df_tarjeta_credito_info.show(5,False)


+---------+-----------------------------------------------------------+------------+------+---------------+--------------+---------------+-------------+--------------+---------------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|CLIENTNUM|Attrition_Flag                                             |Customer_Age|Gender|Dependent_count|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_CoCLIENTNUMunt|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|T

#### describe()

Nos dará información estadística (count, stddev,mean,min,max) del campo numérico que nos interese.

In [ ]:
df_tarjeta_credito_info.describe('Customer_Age').show()

+-------+-----------------+
|summary|     Customer_Age|
+-------+-----------------+
|  count|            10120|
|   mean|           46.325|
| stddev|8.017744397265563|
|    min|               26|
|    max|               73|
+-------+-----------------+



#### select()

La función select nos permite seleccionar las columnas que nos interesen del DF, hay que tener en cuenta que nos devolverá un nuevo DF por lo que si el objetivo es visualizar el resultado tenemos que hacer un show()

In [ ]:
df_tarjeta_credito_info.select('CLIENTNUM','Income_Category').show()

+---------+---------------+
|CLIENTNUM|Income_Category|
+---------+---------------+
|768805383|    $60K - $80K|
|818770008| Less than $40K|
|713982108|   $80K - $120K|
|769911858| Less than $40K|
|709106358|    $60K - $80K|
|713061558|    $40K - $60K|
|810347208|        $120K +|
|818906208|    $60K - $80K|
|710930508|    $60K - $80K|
|719661558|   $80K - $120K|
|708790833|        $120K +|
|710821833|    $40K - $60K|
|710599683|   $80K - $120K|
|816082233|    $60K - $80K|
|712396908| Less than $40K|
|714885258|   $80K - $120K|
|709967358|   $80K - $120K|
|753327333|   $80K - $120K|
|806160108|    $40K - $60K|
|709327383|           null|
+---------+---------------+
only showing top 20 rows



Otra maneras de utilizar select:

In [ ]:
# Notación del punto
df_tarjeta_credito_info\
.select(\
        df_tarjeta_credito_info.Income_Category.alias('Sueldo'),\
        df_tarjeta_credito_info.CLIENTNUM.alias('Cliente'))\
        .show(3)

# Notación de los corchetes
df_tarjeta_credito_info\
.select(\
        df_tarjeta_credito_info["Income_Category"].alias('Sueldo')\
        ,df_tarjeta_credito_info["CLIENTNUM"].alias('Cliente'))\
        .show(3)


# usar sql functions

df_tarjeta_credito_info\
.select(\
        F.col("Income_Category").alias('Sueldo'),\
        F.col("CLIENTNUM").alias('Cliente'))\
        .show(3)


+--------------+---------+
|        Sueldo|  Cliente|
+--------------+---------+
|   $60K - $80K|768805383|
|Less than $40K|818770008|
|  $80K - $120K|713982108|
+--------------+---------+
only showing top 3 rows

+--------------+---------+
|        Sueldo|  Cliente|
+--------------+---------+
|   $60K - $80K|768805383|
|Less than $40K|818770008|
|  $80K - $120K|713982108|
+--------------+---------+
only showing top 3 rows

+--------------+---------+
|        Sueldo|  Cliente|
+--------------+---------+
|   $60K - $80K|768805383|
|Less than $40K|818770008|
|  $80K - $120K|713982108|
+--------------+---------+
only showing top 3 rows



Nos puede interesar conocer los distintos valores que puede contener un campo, para ello podemos utilizar **select().distinct()** o **groupBy()**

In [ ]:
df_tarjeta_credito_info.select('Income_Category').distinct().show()

+---------------+
|Income_Category|
+---------------+
|        $120K +|
|    $60K - $80K|
|           null|
|   $80K - $120K|
|    $40K - $60K|
| Less than $40K|
+---------------+



In [ ]:
df_tarjeta_credito_info.groupBy('Income_Category').count().show()

+---------------+-----+
|Income_Category|count|
+---------------+-----+
|        $120K +|  727|
|    $60K - $80K| 1402|
|           null| 1112|
|   $80K - $120K| 1534|
|    $40K - $60K| 1789|
| Less than $40K| 3561|
+---------------+-----+



### Pre-procesado de datos

#### select()

In [ ]:
df_tarjeta_credito_info = df_tarjeta_credito_info\
                                                  .select(\
                                                          F.col('CLIENTNUM').alias('cliente'),\
                                                          F.col('Customer_Age').alias('edad'),\
                                                          F.col('Gender').alias('genero'),\
                                                          F.col('Income_Category').alias('sueldo'),\
                                                          F.col('Marital_Status').alias('estado civil'),\
                                                          F.col('Months_on_book').alias('meses'),\
                                                          F.col('Credit_Limit')
                                                          
                                                          )

#### Trabajando con columnas

In [ ]:
df_tarjeta_credito_info.show()

+---------+----+------+--------------+------------+-----+------------+
|  cliente|edad|genero|        sueldo|estado civil|meses|Credit_Limit|
+---------+----+------+--------------+------------+-----+------------+
|768805383|  45|     M|   $60K - $80K|     Married|   39|     12691.0|
|818770008|  49|     F|Less than $40K|      Single|   44|      8256.0|
|713982108|  51|     M|  $80K - $120K|     Married|   36|      3418.0|
|769911858|  40|     F|Less than $40K|        null|   34|      3313.0|
|709106358|null|     M|   $60K - $80K|     Married|   21|      4716.0|
|713061558|  44|     M|   $40K - $60K|     Married|   36|      4010.0|
|810347208|  51|     M|       $120K +|     Married|   46|     34516.0|
|818906208|  32|     M|   $60K - $80K|        null|   27|     29081.0|
|710930508|  37|     M|   $60K - $80K|      Single|   36|     22352.0|
|719661558|  48|     M|  $80K - $120K|      Single|   36|     11656.0|
|718813833|null|  null|          null|      Single|   34|     10100.0|
|70879

##### withColumnRenamed()

Nos permite modificar el nombre de las columnas

In [ ]:
df_tarjeta_credito_info = df_tarjeta_credito_info.withColumnRenamed('Credit_Limit','limite de credito')

##### drop()

para elimina columna/s del DF

In [ ]:
df_tarjeta_credito_info = df_tarjeta_credito_info.drop("estado civil","meses")

##### withColumn(nombre_col,col)

  Permite añadir una columna a nuestro dataframe o modificar una existente. 

  Este método nos devuelve un nuevo dataframe, **NO MODIFICA EL ORIGINAL**, si queremos guardar los cambios tenemos que reasignarle el df resultante.

In [ ]:
df_tarjeta_credito_info = df_tarjeta_credito_info\
                                                .withColumn('mayor de edad', F.col('edad')>=18)


In [ ]:
df_tarjeta_credito_info = df_tarjeta_credito_info\
                                                  .withColumn('mayor de edad',(F.col('edad')>=18) & (F.col('sueldo')=='$120K +'))

In [ ]:
df_tarjeta_credito_info = df_tarjeta_credito_info.drop('mayor de edad')

algo un poco más complejo

In [ ]:
df_tarjeta_credito_info.select('sueldo').distinct().show()

+--------------+
|        sueldo|
+--------------+
|       $120K +|
|   $60K - $80K|
|          null|
|  $80K - $120K|
|   $40K - $60K|
|Less than $40K|
+--------------+



In [ ]:
df_categorias = df_tarjeta_credito_info.select('sueldo').distinct()
df_categorias.show()

+--------------+
|        sueldo|
+--------------+
|       $120K +|
|   $60K - $80K|
|          null|
|  $80K - $120K|
|   $40K - $60K|
|Less than $40K|
+--------------+



In [ ]:
df_categorias = df_categorias.withColumn(\
                                          'categoria del cliente',\
                                          F.when( F.col('sueldo').isin('$120K +','$80K - $120K'),'oro')\
                                          .when(F.col('sueldo').isin('$60K - $80K','$40K - $60K'),'plata')\
                                          .when(F.col('sueldo').isin('Less than $40K'),'bronce')\
                                          .otherwise('desconocido')\
                                          )

In [ ]:
df_categorias.show()

+--------------+---------------------+
|        sueldo|categoria del cliente|
+--------------+---------------------+
|       $120K +|                  oro|
|   $60K - $80K|                plata|
|          null|          desconocido|
|  $80K - $120K|                  oro|
|   $40K - $60K|                plata|
|Less than $40K|               bronce|
+--------------+---------------------+



#### Detectar valores missing/nulos

##### isNull()

In [ ]:
df_tarjeta_credito_info.filter((F.col('edad').isNull()) | (F.col('sueldo').isNull()) ).show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|709106358|null|     M|   $60K - $80K|           4716.0|
|710930508|  37|     M|          null|          22352.0|
|718813833|null|  null|          null|          10100.0|
|708790833|null|     M|       $120K +|           6748.0|
|816082233|null|     M|          null|           8547.0|
|712396908|null|     F|Less than $40K|           2436.0|
|714885258|null|     M|  $80K - $120K|           4234.0|
|709327383|  45|     F|          null|          14470.0|
|708300483|  66|     F|          null|           7882.0|
|758551608|null|     M|   $40K - $60K|           6205.0|
|720572508|  38|     F|          null|           9830.0|
|711427458|  44|     F|          null|           6273.0|
|755420433|  59|     F|          null|          10133.0|
|715550508|  45|     F|          null|           2535.0|
|714778833|  49|     F|        

In [ ]:
df_nulls = df_tarjeta_credito_info.limit(20)
df_nulls.show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|818770008|  49|     F|Less than $40K|           8256.0|
|713982108|  51|     M|  $80K - $120K|           3418.0|
|769911858|  40|     F|Less than $40K|           3313.0|
|709106358|null|     M|   $60K - $80K|           4716.0|
|713061558|  44|     M|   $40K - $60K|           4010.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|710930508|  37|     M|          null|          22352.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|718813833|null|  null|          null|          10100.0|
|708790833|null|     M|       $120K +|           6748.0|
|710821833|  65|     M|   $40K - $60K|           9095.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|816082233|null|     M|        

##### na

Con la función `na` (not available) podemos detectar valores missing/nulls en nuestros datos y decidir qué hacer con ellos:  
1. eliminarlos   -> drop()
2. reemplazarlos -> fill()

###### na.drop()

Este método acepta los parámetros:  
    - how    -> any | all  
    - thresh ->  int  
    - subset ->  [lis_of_columns]

- **how** Por defecto how='any' esto elimna todos los registros que contengan en alguno de sus campos un valor null.   
Si pusiéramos how='all' solo eliminearia el registro si todos sus campos fueran null

In [ ]:
df_nulls.na.drop().show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|818770008|  49|     F|Less than $40K|           8256.0|
|713982108|  51|     M|  $80K - $120K|           3418.0|
|769911858|  40|     F|Less than $40K|           3313.0|
|713061558|  44|     M|   $40K - $60K|           4010.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|710821833|  65|     M|   $40K - $60K|           9095.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|709967358|  48|     M|  $80K - $120K|          30367.0|
|753327333|  41|     M|  $80K - $120K|          13535.0|
|806160108|  61|     M|   $40K - $60K|           3193.0|
+---------+----+------+--------------+-----------------+



- **thresh** permite establecer un límite mínimo de campos con valores no-null, un thresh=3 significa q el registro no se borrará si tiene al menos 3 campos con valores no-null 

In [ ]:
df_nulls.na.drop(thresh=3).show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|818770008|  49|     F|Less than $40K|           8256.0|
|713982108|  51|     M|  $80K - $120K|           3418.0|
|769911858|  40|     F|Less than $40K|           3313.0|
|709106358|null|     M|   $60K - $80K|           4716.0|
|713061558|  44|     M|   $40K - $60K|           4010.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|710930508|  37|     M|          null|          22352.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|708790833|null|     M|       $120K +|           6748.0|
|710821833|  65|     M|   $40K - $60K|           9095.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|816082233|null|     M|          null|           8547.0|
|712396908|null|     F|Less tha


- **subset** le pasamos una lista con las columnas a evaluar, si en ese campo el registro tiene un valor null se eliminará el registro por completo.

In [ ]:
df_nulls.na.drop(subset=['sueldo']).show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|818770008|  49|     F|Less than $40K|           8256.0|
|713982108|  51|     M|  $80K - $120K|           3418.0|
|769911858|  40|     F|Less than $40K|           3313.0|
|709106358|null|     M|   $60K - $80K|           4716.0|
|713061558|  44|     M|   $40K - $60K|           4010.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|708790833|null|     M|       $120K +|           6748.0|
|710821833|  65|     M|   $40K - $60K|           9095.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|712396908|null|     F|Less than $40K|           2436.0|
|714885258|null|     M|  $80K - $120K|           4234.0|
|709967358|  48|     M|  $80K -

###### na.fill()

permite sustituir los valores null por un valor que nosotros le pasemos.

 Si pasamos un string solo sustituirá los nulls de los campos del tipo string si pasamos un entero por ejemplo solo los campos de tipo numérico.

In [ ]:
df_nulls.show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|818770008|  49|     F|Less than $40K|           8256.0|
|713982108|  51|     M|  $80K - $120K|           3418.0|
|769911858|  40|     F|Less than $40K|           3313.0|
|709106358|null|     M|   $60K - $80K|           4716.0|
|713061558|  44|     M|   $40K - $60K|           4010.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|710930508|  37|     M|          null|          22352.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|718813833|null|  null|          null|          10100.0|
|708790833|null|     M|       $120K +|           6748.0|
|710821833|  65|     M|   $40K - $60K|           9095.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|816082233|null|     M|        

In [ ]:
df_nulls.na.fill(value=0).show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|818770008|  49|     F|Less than $40K|           8256.0|
|713982108|  51|     M|  $80K - $120K|           3418.0|
|769911858|  40|     F|Less than $40K|           3313.0|
|709106358|   0|     M|   $60K - $80K|           4716.0|
|713061558|  44|     M|   $40K - $60K|           4010.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|710930508|  37|     M|          null|          22352.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|718813833|   0|  null|          null|          10100.0|
|708790833|   0|     M|       $120K +|           6748.0|
|710821833|  65|     M|   $40K - $60K|           9095.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|816082233|   0|     M|        

In [ ]:
df_nulls.na.fill('missing value').show()

+---------+----+-------------+--------------+-----------------+
|  cliente|edad|       genero|        sueldo|limite de credito|
+---------+----+-------------+--------------+-----------------+
|768805383|  45|            M|   $60K - $80K|          12691.0|
|818770008|  49|            F|Less than $40K|           8256.0|
|713982108|  51|            M|  $80K - $120K|           3418.0|
|769911858|  40|            F|Less than $40K|           3313.0|
|709106358|null|            M|   $60K - $80K|           4716.0|
|713061558|  44|            M|   $40K - $60K|           4010.0|
|810347208|  51|            M|       $120K +|          34516.0|
|818906208|  32|            M|   $60K - $80K|          29081.0|
|710930508|  37|            M| missing value|          22352.0|
|719661558|  48|            M|  $80K - $120K|          11656.0|
|718813833|null|missing value| missing value|          10100.0|
|708790833|null|            M|       $120K +|           6748.0|
|710821833|  65|            M|   $40K - 

In [ ]:
df_nulls.na.fill({'edad':0,'genero':'missing','sueldo':'Desconocido'}).show()

+---------+----+-------+--------------+-----------------+
|  cliente|edad| genero|        sueldo|limite de credito|
+---------+----+-------+--------------+-----------------+
|768805383|  45|      M|   $60K - $80K|          12691.0|
|818770008|  49|      F|Less than $40K|           8256.0|
|713982108|  51|      M|  $80K - $120K|           3418.0|
|769911858|  40|      F|Less than $40K|           3313.0|
|709106358|   0|      M|   $60K - $80K|           4716.0|
|713061558|  44|      M|   $40K - $60K|           4010.0|
|810347208|  51|      M|       $120K +|          34516.0|
|818906208|  32|      M|   $60K - $80K|          29081.0|
|710930508|  37|      M|   Desconocido|          22352.0|
|719661558|  48|      M|  $80K - $120K|          11656.0|
|718813833|   0|missing|   Desconocido|          10100.0|
|708790833|   0|      M|       $120K +|           6748.0|
|710821833|  65|      M|   $40K - $60K|           9095.0|
|710599683|  56|      M|  $80K - $120K|          11751.0|
|816082233|   

### operaciones de filtrado de datos

Utilizando la función **filter(condición)** recuperaremos aquellos registros que resuelvan la condición como **True**

In [ ]:
df_tarjeta_credito_info.show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|818770008|  49|     F|Less than $40K|           8256.0|
|713982108|  51|     M|  $80K - $120K|           3418.0|
|769911858|  40|     F|Less than $40K|           3313.0|
|709106358|null|     M|   $60K - $80K|           4716.0|
|713061558|  44|     M|   $40K - $60K|           4010.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|710930508|  37|     M|          null|          22352.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|718813833|null|  null|          null|          10100.0|
|708790833|null|     M|       $120K +|           6748.0|
|710821833|  65|     M|   $40K - $60K|           9095.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|816082233|null|     M|        

In [ ]:
df_tarjeta_credito_info.filter(F.col('limite de credito')>10000).show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|710930508|  37|     M|          null|          22352.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|718813833|null|  null|          null|          10100.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|709967358|  48|     M|  $80K - $120K|          30367.0|
|753327333|  41|     M|  $80K - $120K|          13535.0|
|709327383|  45|     F|          null|          14470.0|
|806165208|  47|     M|   $60K - $80K|          20979.0|
|789124683|  54|     M|  $80K - $120K|          12217.0|
|720466383|  59|     M|   $40K - $60K|          14784.0|
|804424383|  63|     M|   $60K - $80K|          10215.0|
|787937058|  58|     M|  $80K -

In [ ]:
df_tarjeta_credito_info.filter(F.col('limite de credito').between(10000,15000)).show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|718813833|null|  null|          null|          10100.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|753327333|  41|     M|  $80K - $120K|          13535.0|
|709327383|  45|     F|          null|          14470.0|
|789124683|  54|     M|  $80K - $120K|          12217.0|
|720466383|  59|     M|   $40K - $60K|          14784.0|
|804424383|  63|     M|   $60K - $80K|          10215.0|
|787937058|  58|     M|  $80K - $120K|          12555.0|
|708155733|  53|     M|       $120K +|          14734.0|
|755420433|  59|     F|          null|          10133.0|
|719720058|  44|     M|   $60K - $80K|          12756.0|
|717929133|  44|     M|   $60K - $80K|          14450.0|
|711844758|  48|     M|  $80K -

Podemos filtrar por múltiples condiciones, cada una de las condicones tiene que estar dentro de paréntesis y utilizar un operador lógico (&, '|', '~') para concatenarlas.

In [ ]:
df_tarjeta_credito_info.filter( (F.col('limite de credito')>10000) & (F.col('genero')=='F')).show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|709327383|  45|     F|          null|          14470.0|
|785432733|  42|     F|Less than $40K|          15433.0|
|773146383|  50|     F|   $40K - $60K|          17304.0|
|755420433|  59|     F|          null|          10133.0|
|714778833|  49|     F|          null|          23032.0|
|713843283|  41|     F|          null|          34516.0|
|710806083|  50|     F|Less than $40K|          11888.0|
|708296883|  44|     F|   $40K - $60K|          11749.0|
|712215258|  50|     F|   $40K - $60K|          18386.0|
|718759833|  44|     F|          null|          32643.0|
|716493408|  45|     F|          null|          15875.0|
|714964758|  45|     F|   $40K - $60K|          10400.0|
|712977183|  57|     F|   $40K - $60K|          19482.0|
|721148358|  49|     F|          null|          12860.0|
|779956608|  41|     F|        

In [ ]:
df_tarjeta_credito_info\
                        .filter( (F.col('limite de credito')>10000) & (F.col('genero')=='F'))\
                        .select('cliente','sueldo')\
                        .na.drop()\
                        .show()

+---------+--------------+
|  cliente|        sueldo|
+---------+--------------+
|785432733|Less than $40K|
|773146383|   $40K - $60K|
|710806083|Less than $40K|
|708296883|   $40K - $60K|
|712215258|   $40K - $60K|
|714964758|   $40K - $60K|
|712977183|   $40K - $60K|
|711441333|Less than $40K|
|803043933|Less than $40K|
|711297483|Less than $40K|
|716455683|Less than $40K|
|710865633|Less than $40K|
|712981383|Less than $40K|
|716302383|   $40K - $60K|
|714513033|   $40K - $60K|
|717956283|   $40K - $60K|
|708177333|Less than $40K|
|718206783|Less than $40K|
|713378133|Less than $40K|
|720650208|   $40K - $60K|
+---------+--------------+
only showing top 20 rows



### Agrupación y funciones de agregación

El groupBy y las funciones de agregación funcionan juntas.

Las funciones de agregación básicas son:

- sum
- mean
- count
- min
- max



In [ ]:
df_tarjeta_credito_info.show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|818770008|  49|     F|Less than $40K|           8256.0|
|713982108|  51|     M|  $80K - $120K|           3418.0|
|769911858|  40|     F|Less than $40K|           3313.0|
|709106358|null|     M|   $60K - $80K|           4716.0|
|713061558|  44|     M|   $40K - $60K|           4010.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|710930508|  37|     M|          null|          22352.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|718813833|null|  null|          null|          10100.0|
|708790833|null|     M|       $120K +|           6748.0|
|710821833|  65|     M|   $40K - $60K|           9095.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|816082233|null|     M|        

#### sum()   

Podemos especificar un campo en el sum pero si no lo hacemos hará el sumatorio de todos los campos numéricos

In [ ]:
df_tarjeta_credito_info.groupBy('genero').sum().orderBy('sum(limite de credito)', ascending=False).show()

+------+-------------+---------+----------------------+
|genero| sum(cliente)|sum(edad)|sum(limite de credito)|
+------+-------------+---------+----------------------+
|     M|3527394286161|   219915|  6.0473500899999894E7|
|     F|3956028957831|   248815|   2.690771120000024E7|
|  null|    718813833|     null|               10100.0|
+------+-------------+---------+----------------------+



In [ ]:
df_tarjeta_credito_info.groupBy('genero').sum('limite de credito').orderBy('sum(limite de credito)', ascending=False).show()

+------+----------------------+
|genero|sum(limite de credito)|
+------+----------------------+
|     M|  6.0473500899999894E7|
|     F|   2.690771120000024E7|
|  null|               10100.0|
+------+----------------------+



#### mean()

Nos da el valor medio de un campo

In [ ]:
df_tarjeta_credito_info.groupBy('genero').mean('limite de credito').orderBy('avg(limite de credito)', ascending=False).show()

+------+----------------------+
|genero|avg(limite de credito)|
+------+----------------------+
|     M|     12685.86131739037|
|  null|               10100.0|
|     F|     5022.906701512085|
+------+----------------------+



#### count()

In [ ]:
df_tarjeta_credito_info.groupBy('genero').count().orderBy('count', ascending=False).show()

+------+-----+
|genero|count|
+------+-----+
|     F| 5357|
|     M| 4767|
|  null|    1|
+------+-----+



#### max() / min()

Nos devuelvo el valor máximo de todos los campos numéricos del DF

In [ ]:
df_tarjeta_credito_info.groupBy('genero').max().orderBy('max(limite de credito)', ascending=False).show()

+------+------------+---------+----------------------+
|genero|max(cliente)|max(edad)|max(limite de credito)|
+------+------------+---------+----------------------+
|     M|   828298908|       73|               34516.0|
|     F|   828343083|       67|               34516.0|
|  null|   718813833|     null|               10100.0|
+------+------------+---------+----------------------+



In [ ]:
df_tarjeta_credito_info.groupBy('genero').min().orderBy('min(limite de credito)', ascending=False).show()

+------+------------+---------+----------------------+
|genero|min(cliente)|min(edad)|min(limite de credito)|
+------+------------+---------+----------------------+
|  null|   718813833|     null|               10100.0|
|     F|   708082083|       26|                1438.3|
|     M|   708083283|       26|                1438.3|
+------+------------+---------+----------------------+



#### agg()

Nos permite usar funciones de agregación directamente sin usar previamente un groupBy()

In [ ]:
df_tarjeta_credito_info\
                      .groupBy('genero')\
                      .agg(\
                          F.count('genero').alias('numero de clientes'),\
                          F.max('limite de credito').alias('límite de crédito max'),\
                          F.min('limite de credito').alias('límite de crédito min'),\
                          F.avg('limite de credito').alias('media de crédito')\
                          )\
                          .orderBy('media de crédito', ascending=False)\
                          .show()

+------+------------------+---------------------+---------------------+-----------------+
|genero|numero de clientes|límite de crédito max|límite de crédito min| media de crédito|
+------+------------------+---------------------+---------------------+-----------------+
|     M|              4767|              34516.0|               1438.3|12685.86131739037|
|  null|                 0|              10100.0|              10100.0|          10100.0|
|     F|              5357|              34516.0|               1438.3|5022.906701512085|
+------+------------------+---------------------+---------------------+-----------------+



In [ ]:
df_tarjeta_credito_info\
                        .groupBy('genero')\
                        .agg({\
                              'limite de credito':'max',\
                              'edad':'min',\
                              'cliente':'count'\
                              })\
                        .show()

+------+--------------+---------+----------------------+
|genero|count(cliente)|min(edad)|max(limite de credito)|
+------+--------------+---------+----------------------+
|     F|          5357|       26|               34516.0|
|  null|             1|     null|               10100.0|
|     M|          4767|       26|               34516.0|
+------+--------------+---------+----------------------+



---

In [ ]:
max_limit = df_tarjeta_credito_info.agg({'limite de credito':'max'}).collect()[0]['max(limite de credito)']


df_tarjeta_credito_info\
                        .filter(F.col('limite de credito')==max_limit)\
                        .groupBy(F.col('genero'))\
                        .agg(\
                              F.count('genero').alias('num clientes'),\
                              F.max('limite de credito').alias('límite de crédito max')\
                              ).show()


+------+------------+---------------------+
|genero|num clientes|límite de crédito max|
+------+------------+---------------------+
|     F|          40|              34516.0|
|     M|         468|              34516.0|
+------+------------+---------------------+



In [ ]:
df_tarjeta_credito_info\
                        .withColumn(\
                                    'ratio credito', \
                                    (F.col('limite de credito')/df_tarjeta_credito_info.agg({'limite de credito':'max'}).collect()[0]['max(limite de credito)'])*100\
                                    ).show()

+---------+----+------+--------------+-----------------+------------------+
|  cliente|edad|genero|        sueldo|limite de credito|     ratio credito|
+---------+----+------+--------------+-----------------+------------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|36.768455209178356|
|818770008|  49|     F|Less than $40K|           8256.0|23.919341754548615|
|713982108|  51|     M|  $80K - $120K|           3418.0| 9.902653841696605|
|769911858|  40|     F|Less than $40K|           3313.0| 9.598447096998493|
|709106358|null|     M|   $60K - $80K|           4716.0| 13.66322864758373|
|713061558|  44|     M|   $40K - $60K|           4010.0|11.617800440375477|
|810347208|  51|     M|       $120K +|          34516.0|             100.0|
|818906208|  32|     M|   $60K - $80K|          29081.0| 84.25367945300731|
|710930508|  37|     M|          null|          22352.0| 64.75837292849693|
|719661558|  48|     M|  $80K - $120K|          11656.0|33.769845868582685|
|718813833|n

### Cruzando Dataframes


#### joins



In [ ]:
df_tarjeta_credito_info.show()
df_tarjeta_credito_estudios.show()

+---------+----+------+--------------+-----------------+
|  cliente|edad|genero|        sueldo|limite de credito|
+---------+----+------+--------------+-----------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|
|818770008|  49|     F|Less than $40K|           8256.0|
|713982108|  51|     M|  $80K - $120K|           3418.0|
|769911858|  40|     F|Less than $40K|           3313.0|
|709106358|null|     M|   $60K - $80K|           4716.0|
|713061558|  44|     M|   $40K - $60K|           4010.0|
|810347208|  51|     M|       $120K +|          34516.0|
|818906208|  32|     M|   $60K - $80K|          29081.0|
|710930508|  37|     M|          null|          22352.0|
|719661558|  48|     M|  $80K - $120K|          11656.0|
|718813833|null|  null|          null|          10100.0|
|708790833|null|     M|       $120K +|           6748.0|
|710821833|  65|     M|   $40K - $60K|           9095.0|
|710599683|  56|     M|  $80K - $120K|          11751.0|
|816082233|null|     M|        

In [ ]:
# inner join

df_tarjeta_credito_info.join(df_tarjeta_credito_estudios,\
                                                        on=[df_tarjeta_credito_info.cliente==df_tarjeta_credito_estudios.CLIENTNUM],\
                                                        how='inner')\
                                                        .select('*').show(5)

+---------+----+------+--------------+-----------------+---------+---------------+
|  cliente|edad|genero|        sueldo|limite de credito|CLIENTNUM|Education_Level|
+---------+----+------+--------------+-----------------+---------+---------------+
|768805383|  45|     M|   $60K - $80K|          12691.0|768805383|    High School|
|818770008|  49|     F|Less than $40K|           8256.0|818770008|       Graduate|
|713982108|  51|     M|  $80K - $120K|           3418.0|713982108|       Graduate|
|769911858|  40|     F|Less than $40K|           3313.0|769911858|    High School|
|709106358|null|     M|   $60K - $80K|           4716.0|709106358|     Uneducated|
+---------+----+------+--------------+-----------------+---------+---------------+
only showing top 5 rows



In [ ]:
# leftanti -> nos devuelve los registros de la tabla de la izq q NO hacen match (solo campos de la tabla de la izq)

df_tarjeta_credito_info.join(df_tarjeta_credito_estudios,\
                                                        on=[df_tarjeta_credito_info.cliente==df_tarjeta_credito_estudios.CLIENTNUM],\
                                                        how='leftanti').show()

+---------+----+------+------------+-----------------+
|  cliente|edad|genero|      sueldo|limite de credito|
+---------+----+------+------------+-----------------+
|778348233|  53|     M|$80K - $120K|           2753.0|
|771490833|  52|     M|$80K - $120K|           4745.0|
+---------+----+------+------------+-----------------+

